In [ ]:
# Setup Spark Session
from pyspark.sql import SparkSession

# Import API Modules 
from flask import Flask, request, jsonify
from flask_cors import CORS
import datetime

# Import Translator
from googletrans import Translator

# Impport Tokenizer and Pipeline Tools
from transformers import DistilBertTokenizer
from pyspark.ml.pipeline import PipelineModel

# Setup Spark Session
sc = SparkSession \
        .builder \
        .master("local[*]") \
        .appName("API-Demo") \
        .getOrCreate()

# Add Numerical Label for Emotions
emotion_key = {
    "boredom": 0,
    "love": 1,
    "relief": 2,
    "fun": 3,
    "hate": 4,
    "neutral": 5,
    "anger": 6,
    "happiness": 7,
    "surprise": 8,
    "sadness": 9,
    "worry": 10,
    "enthusiasm": 11,
    "empty": 12,
    "---": 13
}

# Initialize the AutoTokenizer
# DistilBERT is a compact version of the BERT (Bidirectional Encoder Representations from Transformers) 
# We are pulling this from Huggingface via the Huggingface transformers
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-multilingual-cased")

# Load Saved Models from the "2 - ML Training And Testing.ipynb" file
model_en_rfc = PipelineModel.load("./models/model_en_rfc.model")
model_en_nb = PipelineModel.load("./models/model_en_nb.model")
model_en_lr = PipelineModel.load("./models/model_en_lr.model")
model_de_rfc = PipelineModel.load("./models/model_de_rfc.model")
model_de_nb = PipelineModel.load("./models/model_de_nb.model")
model_de_lr = PipelineModel.load("./models/model_de_lr.model")


app = Flask(__name__)
CORS(app)  # Use the CORS class to enable CORS for the entire app
translator = Translator()

# Setup Emotion Key Reverse
emotions = {v: k for k, v in emotion_key.items()}

@app.route('/predict', methods=['GET'])
def transform_data():
    start_time = datetime.datetime.now()
    sentence_en = request.args.get('sentence_en')
    sentence_de = request.args.get('sentence_de')
    
    print(f"English: {sentence_en}")
    print(f"German: {sentence_de}")
    
    if sentence_en:
        sentence_de = translator.translate(sentence_en, dest="de").text
    elif sentence_de:
        sentence_en = translator.translate(sentence_de, dest="en").text
    else:
        return jsonify({'error': 'No Sentence provided!'})
    
    print(f"English: {sentence_en}")
    print(f"German: {sentence_de}")
    
    words_en = tokenizer.tokenize(sentence_en)
    words_de = tokenizer.tokenize(sentence_de)

    # Create Englsih & German Dataframe
    df_en_api = sc.createDataFrame([(sentence_en, words_en)], ["sentence", "words"])
    df_de_api = sc.createDataFrame([(sentence_de, words_de)], ["sentence", "words"])
    
    predictions_en_rfc = model_en_rfc.transform(df_en_api)
    predictions_en_nb = model_en_nb.transform(df_en_api)
    predictions_en_lr = model_en_lr.transform(df_en_api)
    predictions_de_rfc = model_de_rfc.transform(df_de_api)
    predictions_de_nb = model_de_nb.transform(df_de_api)
    predictions_de_lr = model_de_lr.transform(df_de_api)
    
    # Get End Time
    end_time = datetime.datetime.now()
    elapsed_time = (end_time - start_time).total_seconds()
    
    #Build Response
    resp = {
        "sentence": {
            "english": sentence_en,
            "german": sentence_de,
        },
        "predictions": {
            "en_rfc": emotions.get(predictions_en_rfc.collect()[0]["prediction"]),
            "en_nb":  emotions.get(predictions_en_nb.collect()[0]["prediction"]),
            "en_lr":  emotions.get(predictions_en_lr.collect()[0]["prediction"]),
            "de_rfc": emotions.get(predictions_de_rfc.collect()[0]["prediction"]),
            "de_nb":  emotions.get(predictions_de_nb.collect()[0]["prediction"]),
            "de_lr":  emotions.get(predictions_de_lr.collect()[0]["prediction"]),
        },
        "metadata": {
            "spark": sc.version,
            "datetime": {
                "start": start_time,
                "end": end_time,
                "elapsedInSeconds": elapsed_time,
            }
        }
    }
    
    return jsonify(resp)

if __name__ == '__main__':
    app.run(host='127.0.0.1', port=8080) # Make sure we listen on ":8080" since Julyter run on ":8888"


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/08/14 15:23:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit


English: I am so happy about this news!
German: 
English: I am so happy about this news!
German: Ich freue mich sehr über diese Neuigkeiten!
23/08/14 15:24:58 WARN DAGScheduler: Broadcasting large task binary with size 5.2 MiB
23/08/14 15:24:58 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
23/08/14 15:24:59 WARN DAGScheduler: Broadcasting large task binary with size 5.5 MiB


127.0.0.1 - - [14/Aug/2023 15:24:59] "GET /predict?sentence_en=I%20am%20so%20happy%20about%20this%20news!&sentence_de= HTTP/1.1" 200 -


...

127.0.0.1 - - [14/Aug/2023 15:26:31] "GET /predict?sentence_en=This%20not%20a%20good%20idea,%20we%20should%20stop.&sentence_de= HTTP/1.1" 200 -


English: This is bad idea and we need to stop right now
German: 
English: This is bad idea and we need to stop right now
German: Dies ist eine schlechte Idee und wir müssen jetzt aufhören
23/08/14 15:26:45 WARN DAGScheduler: Broadcasting large task binary with size 5.2 MiB
23/08/14 15:26:45 WARN DAGScheduler: Broadcasting large task binary with size 5.5 MiB


127.0.0.1 - - [14/Aug/2023 15:26:45] "GET /predict?sentence_en=This%20is%20bad%20idea%20and%20we%20need%20to%20stop%20right%20now&sentence_de= HTTP/1.1" 200 -


English: Gosh I hate doing this today.
German: 
English: Gosh I hate doing this today.
German: Meine Güte, ich hasse es heute.
23/08/14 15:54:41 WARN DAGScheduler: Broadcasting large task binary with size 5.2 MiB
23/08/14 15:54:41 WARN DAGScheduler: Broadcasting large task binary with size 5.5 MiB


127.0.0.1 - - [14/Aug/2023 15:54:42] "GET /predict?sentence_en=Gosh%20I%20hate%20doing%20this%20today.&sentence_de= HTTP/1.1" 200 -
